Выполнила Леопаева Л.А., 6402-010302D

Лабораторная работа 4

In [1]:
import random
import numpy as np

Часть 1

4.1. Функция формирования порождающей и проверочной
матриц расширенного кода Голея (24,12,8).

In [2]:
# Матрица Б (часть расширенного кода Голея)
B = np.array([
    [1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
    [1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1],
    [0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1],
    [1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1],
    [1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1],
    [1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1],
    [0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1],
    [0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1],
    [0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1],
    [1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1],
    [0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
])

In [3]:
def generate_g_h(B):
    G = np.hstack((np.eye(12, dtype=int), B))
    H = np.vstack((np.eye(12, dtype=int), B))
    return G, H

4.2. Исследование расширенного кода Голея для одно-, двух-,
трёх- и четырёхкратных ошибок.

In [4]:
def simulate_errors(word, G, count):
    encrypted_word = word @ G % 2
    print(f"\nИсходное слово: {word}")
    print(f"Зашифрованное слово: {encrypted_word}")
    error_positions = random.sample(range(encrypted_word.shape[0]), count)
    error_vector = np.zeros(encrypted_word.shape[0], dtype=int)
    for idx in error_positions:
        error_vector[idx] = 1
    word_with_error = (encrypted_word + error_vector) % 2
    print(f"Слово с {count} ошибками: {word_with_error}")
    return word_with_error

In [5]:
def find_and_correct_errors(word_with_error, H, B):
    syndrome = word_with_error @ H % 2
    corrected = None
    if np.sum(syndrome) <= 3:
        corrected = np.hstack((syndrome, np.zeros(len(syndrome), dtype=int)))
    else:
        for i in range(len(B)):
            temp_syndrome = (syndrome + B[i]) % 2
            if np.sum(temp_syndrome) <= 2:
                mistake_index = np.zeros(len(syndrome), dtype=int)
                mistake_index[i] = 1
                corrected = np.hstack((temp_syndrome, mistake_index))
    if corrected is None:
        syndrome_B = syndrome @ B % 2
        if np.sum(syndrome_B) <= 3:
            corrected = np.hstack((np.zeros(len(syndrome), dtype=int), syndrome_B))
        else:
            for i in range(len(B)):
                temp_syndrome = (syndrome_B + B[i]) % 2
                if np.sum(temp_syndrome) <= 2:
                    mistake_index = np.zeros(len(syndrome), dtype=int)
                    mistake_index[i] = 1
                    corrected = np.hstack((mistake_index, temp_syndrome))
    return corrected

In [6]:
def correct_received_word(original_word, received_word, H, B, G):
    corrected = find_and_correct_errors(received_word, H, B)
    if corrected is None:
        print("Ошибка не может быть исправлена!")
        return
    corrected_word = (received_word + corrected) % 2
    print(f"Исправленное слово: {corrected_word}")
    encoded_word = original_word @ G % 2
    if not np.array_equal(encoded_word, corrected_word):
        print("Сообщение не было декодировано правильно!")

In [9]:
def golay_code_research():
    G, H = generate_g_h(B)
    print(f"G:\n{G}\nH:\n{H}")

    word = np.array([i % 2 for i in range(len(G))])

    for i in range(5):
        received_word = simulate_errors(word, G, i)
        correct_received_word(word, received_word, H, B, G)
        print('')

In [10]:
golay_code_research()

G:
[[1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 1 0 0 0 1 0 1]
 [0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 0 0 1 0 1 1]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 1 1 1]
 [0 0 0 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 1 1 0 1]
 [0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 0 1 1 0 1 1]
 [0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 1 1 1]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 1 0 1 1 1 1]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 1 1 1 0 1]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 1 0 1 1 1 0 0 1]
 [0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 1 0 1 1 1 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 1 1 1 0 0 0 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0]]
H:
[[1 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 1]
 [1 1 0 1 1 1 0 0 0 1 0 1]
 [1 0 1 1 1 0 0 0 1 0 1 1]
 [0

Часть 2

4.3. Функция формирования порождающей и проверочных
матриц кода Рида-Маллера

In [11]:
def generate_rm_matrices(r, m):
    if 0 < r < m:
        upper_left = generate_rm_matrices(r, m - 1)
        lower_right = generate_rm_matrices(r - 1, m - 1)
        return np.hstack([np.vstack([upper_left, np.zeros((len(lower_right), len(upper_left.T)), int)]), \
                          np.vstack([upper_left, lower_right])])
    elif r == 0:
        return np.ones((1, 2 ** m), dtype=int)
    elif r == m:
        upper = generate_rm_matrices(m - 1, m)
        lower = np.zeros((1, 2 ** m), dtype=int)
        lower[0][len(lower.T) - 1] = 1
        return np.vstack([upper, lower])

In [12]:
def generate_hm_matrices(i, m):
    H = np.array([[1, 1], [1, -1]])
    result = np.kron(np.eye(2 ** (m - i)), H)
    result = np.kron(result, np.eye(2 ** (i - 1)))
    return result

4.4-5. Исследование кода Рида-Маллера для одно- и
двукратных ошибок.

In [13]:
def rm_code_research(word, G, error_count, m):
    received_word = simulate_errors(word, G, error_count)
    for i in range(len(received_word)):
        if received_word[i] == 0:
            received_word[i] = -1
    w_t = [received_word @ generate_hm_matrices(1, m)]
    for i in range(2, m + 1):
        w_t.append(w_t[-1] @ generate_hm_matrices(i, m))
    maximum = w_t[0][0]
    index = -1
    for i in range(len(w_t)):
        for j in range(len(w_t[i])):
            if abs(w_t[i][j]) > abs(maximum):
                index = j
                maximum = w_t[i][j]
    counter = 0
    for i in range(len(w_t)):
        for j in range(len(w_t[i])):
            if abs(w_t[i][j]) == abs(maximum):
                counter += 1
            if counter > 1:
                print("Невозможно исправить ошибку.\n")
                return
    corrected_word = list(map(int, list(('{' + f'0:0{m}b' + '}').format(index))))
    if maximum > 0:
        corrected_word.append(1)
    else:
        corrected_word.append(0)
    print(f"Исправленное сообщение: {np.array(corrected_word[::-1])}")

In [14]:
def reed_muller_code_research():
    m = 3
    print(f"\nПорождающая матрица Рида-Маллера (1,3): \n{generate_rm_matrices(1, m)}\n")
    word = np.array([i % 2 for i in range(len(generate_rm_matrices(1, m)))])
    for j in range(1, 3):
        rm_code_research(word, generate_rm_matrices(1, m), j, m)

    m = 4
    print(f"\nПорождающая матрица Рида-Маллера (1,4)\n{generate_rm_matrices(1, m)}\n")
    word = np.array([i % 2 for i in range(len(generate_rm_matrices(1, m)))])
    for j in range(1, 5):
        rm_code_research(word, generate_rm_matrices(1, m), j, m)

In [15]:
reed_muller_code_research()


Порождающая матрица Рида-Маллера (1,3): 
[[1 1 1 1 1 1 1 1]
 [0 1 0 1 0 1 0 1]
 [0 0 1 1 0 0 1 1]
 [0 0 0 0 1 1 1 1]]


Исходное слово: [0 1 0 1]
Зашифрованное слово: [0 1 0 1 1 0 1 0]
Слово с 1 ошибками: [0 1 0 1 1 1 1 0]
Исправленное сообщение: [0 1 0 1]

Исходное слово: [0 1 0 1]
Зашифрованное слово: [0 1 0 1 1 0 1 0]
Слово с 2 ошибками: [0 1 0 0 1 0 0 0]
Невозможно исправить ошибку.


Порождающая матрица Рида-Маллера (1,4)
[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
 [0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1]
 [0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1]
 [0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1]]


Исходное слово: [0 1 0 1 0]
Зашифрованное слово: [0 1 0 1 1 0 1 0 0 1 0 1 1 0 1 0]
Слово с 1 ошибками: [0 1 0 1 1 0 1 0 0 1 0 1 0 0 1 0]
Исправленное сообщение: [0 1 0 1 0]

Исходное слово: [0 1 0 1 0]
Зашифрованное слово: [0 1 0 1 1 0 1 0 0 1 0 1 1 0 1 0]
Слово с 2 ошибками: [0 1 0 1 1 0 1 1 0 0 0 1 1 0 1 0]
Исправленное сообщение: [0 1 0 1 0]

Исходное слово: [0 1 0 1 0]
Зашифрованное 